# Tumor Growth Model Analysis

This notebook analyzes and improves the tumor growth model for estimating primary tumor expansion age. We'll:
1. Analyze the current model and data
2. Implement an improved model
3. Perform sensitivity analysis
4. Explore additional analytical approaches

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [ ]:
# Set plotting style
#plt.style.use('seaborn-white')
sns.set_theme(style="whitegrid") 
sns.set_palette("CMRmap")

# Update plotting parameters
plt.rcParams['figure.dpi'] = 300
sns.set(rc={'figure.dpi': 300})
#plt.rcParams['axes.grid'] = True  # Remove gridlines
#plt.rcParams['axes.spines.right'] = True  # Remove right spine
#plt.rcParams['axes.spines.top'] = False   # Remove top spine
plt.rcParams['axes.facecolor'] = 'white'  # Set white background

## 1. Data Loading and Initial Analysis

We have loaded data from **eight** studies containing cohort-level data about the growth characteristics of pheochromocytomas and paragangliomas. The studies are:
* Tamaki et al (2022). PMID: 34972002
* Michałowska et al (2017). PMID: 27967220
* Jansen et al (2000). PMID: 10870065
* Walther et al (1999). PMID: 10458336
* Wang et al (2016). PMID: 27073708
* Heesterman et al (2017). PMID: 29134169
* Tooker et al (2023). PMID: 37590887
* Carlson et al (2015). PMID: 25361637

Let's begin with a quick exploratory analysis of the basic variables in the data.

In [ ]:
# Load data
dt = pd.read_excel('../../metadata/PPGL_doubling_time_estimates.xlsx', sheet_name='doubling_time')
diam = pd.read_excel('../../metadata/PPGL_doubling_time_estimates.xlsx', sheet_name='volume')

# Calculate weighted statistics for doubling time
weights_dt = dt['num_tumors'] / dt['num_tumors'].sum()

dt_min = dt['doubling_time'].min() * 365
dt_max = dt['doubling_time'].max() * 365
dt_median = np.average(dt['doubling_time'], weights=weights_dt) * 365

# Calculate weighted quartiles for doubling time
def weighted_quantile(values, weights, q):
    indices = np.argsort(values)
    sorted_values = values[indices]
    sorted_weights = weights[indices]
    cumsum = np.cumsum(sorted_weights)
    cumsum = cumsum / cumsum[-1]
    return np.interp(q, cumsum, sorted_values)

dt_1qt = weighted_quantile(dt['doubling_time'].values, weights_dt.values, 0.25) * 365
dt_3qt = weighted_quantile(dt['doubling_time'].values, weights_dt.values, 0.75) * 365

# Constants
limit_size = 1e11
log_limit_size = np.log(limit_size)

# Calculate weighted statistics for volume
weights_vol = diam['num_tumors'] / diam['num_tumors'].sum()
vol = np.average(diam['volume_cm'], weights=weights_vol)
vol_min = diam['volume_cm'].min()
vol_max = diam['volume_cm'].max()

vol_1qt = weighted_quantile(diam['volume_cm'].values, weights_vol.values, 0.25)
vol_3qt = weighted_quantile(diam['volume_cm'].values, weights_vol.values, 0.75)

# Convert volumes to sizes
dx_size = vol * 1e8
dx_size_min = vol_min * 1e8
dx_size_max = vol_max * 1e8
dx_size_1q = vol_1qt * 1e8
dx_size_3q = vol_3qt * 1e8

In [ ]:
def calculate_growth_parameters(time_vals, size_vals):
    """Calculate growth parameters for combinations of time and size values"""
    results = []
    
    for time in time_vals:
        for size in size_vals:
            try:
                # Calculate beta
                beta = 1/time * np.log(
                    (log_limit_size - np.log(size)) / 
                    (log_limit_size - np.log(2 * size))
                )
                
                # Calculate alpha
                alpha = log_limit_size * beta
                
                # Calculate expected age
                exp_age = -1 / beta * np.log(1 - np.log(size) * beta / alpha) / 365
                
                if exp_age < 100:  # Filter ages greater than lifespan
                    results.append({
                        'time': time,
                        'size': size,
                        'beta': beta,
                        'alpha': alpha,
                        'age': exp_age
                    })
            except:
                continue
    
    return pd.DataFrame(results)


def calculate_trajectories(df, alpha_col='alpha', beta_col='beta', age_col='age'):
    """
    Calculate growth trajectories for each parameter combination.

    Parameters:
    - df (DataFrame): Input dataframe containing the required columns.
    - exp_age_col (str): Column name for expected age.
    - alpha_col (str): Column name for alpha parameter.
    - beta_col (str): Column name for beta parameter.
    - age_col (str): Column name for age parameter.

    Returns:
    - DataFrame: Concatenated DataFrame with growth trajectories.
    """
    trajectories = []
    
    for _, row in df.iterrows():
        time_steps = np.arange(0, 90, 0.01)
        time_steps2 = time_steps - row[age_col]
        
        # Calculate St (size at time t)
        St = np.exp(row[alpha_col] / row[beta_col] * 
                    (1 - np.exp(-row[beta_col] * time_steps * 365)))
        
        trajectory_df = pd.DataFrame({
            'time_step': time_steps,
            'time_step2': time_steps2,
            'St': St,
            'alpha': row[alpha_col],
            'beta': row[beta_col],
            'exp_age': row[age_col]
        })
        
        trajectories.append(trajectory_df)
    
    return pd.concat(trajectories)


In [ ]:
# Set up parameters for both models
time_vals = np.array([dt_1qt, dt_median, dt_3qt])
size_vals = np.array([dx_size_min, dx_size_1q, dx_size, dx_size_3q, dx_size_max])

# Calculate results for the weighted model
df_weighted = calculate_growth_parameters(time_vals, size_vals)
df_weighted.head(10)

In [ ]:
trajectories_weighted = calculate_trajectories(df_weighted)
trajectories_filtered = trajectories_weighted[trajectories_weighted['time_step2'] < 0]

# Plotting
plt.figure(figsize=(12, 6))

# Plot trajectories
for exp_age in df_weighted['age'].unique():
    mask = trajectories_filtered['exp_age'] == exp_age
    plt.plot(trajectories_filtered[mask]['time_step2'],
             np.log10(trajectories_filtered[mask]['St']),
             color='#3288bd', alpha=0.6)

plt.axvline(x=0, color='black', linestyle='-')
plt.axvline(x=df_weighted['age'].mean()*-1, color='black', linestyle='--')
plt.ylim(0, 10)
plt.xlim(-90, 0)

plt.title('Tumor Growth Trajectories')
plt.xlabel('Time relative to diagnosis (years)')
plt.ylabel('log10(Tumor Size)')

plt.grid(True, which='major', axis='both', color='black', alpha=0.1)
plt.show()

## 2. Modeling the Primary Tumor Expansion Age w/ Patient Level Data 

In [ ]:
# Load the data
df = pd.read_excel("../../metadata/PPGL_doubling_time_estimates.xlsx", sheet_name='tumors')
df.head(10)

We have loaded data from **four** studies containing tumor-level data about the growth characteristics of pheochromocytomas and paragangliomas. The studies this data was extracted from include:
* Michałowska et al (2017). **PMID:** 27967220
* Wang et al (2016). **PMID:** 27073708
* Tooker et al (2023). **PMID:** 37590887
* Carlson et al (2015). **PMID:** 25361637

Let's begin with a quick exploratory analysis of the basic variables in the data.

In [ ]:
df.groupby(['germline'])['germline'].count() # Distribution of germline mutations. 91 patients

In [ ]:
# Plot distrubtion of ages
plt.figure(figsize=(12, 6))
sns.histplot(df['age'])
plt.xlabel('Patient Age')
plt.ylabel('Count')
plt.title('Distribution of Patient Ages')
plt.grid(True)
plt.show()

In [ ]:
# Plot distribution of percent volume growth
plt.figure(figsize=(12, 6))
sns.histplot(df['vol_per_change'])
plt.xlabel('Percent Change in Volume of Tumors')
plt.ylabel('Count')
plt.title('Distribution of Percent Changes')
plt.grid(True)
plt.show()

We can see that several patients have no growth or negative growth over their study period. These patients will have to be excluded from further analysis.

In [ ]:
print(len(df[df['vol_per_change'] > 0.0]), " patients have growth during their study periods")
print(len(df[df['vol_per_change'] < 0.0]), " patients have no growth or negative growth during their study periods")

In [ ]:
df = df[df['vol_per_change'] > 0.0]
df.groupby(['germline'])['germline'].count() # Distribution of germline mutations

## 2. Implement Analysis Utilized by Hu et al (Nature Genetics 2020)

In [ ]:
def calculate_primary_tumor_expansion_age(row):
    """Implement the original model calculations"""
    log_limit_size = np.log(1e11)
    dx_size = row['tumor_size_t1'] * 1e8
    doubling_time_days = row['doubling_time_yr'] * 365
    
    try:
        beta = 1/doubling_time_days * np.log(
            (log_limit_size - np.log(dx_size)) / 
            (log_limit_size - np.log(2 * dx_size))
        )
        alpha = log_limit_size * beta
        exp_age = -1/beta * np.log(1 - np.log(dx_size) * beta / alpha) / 365
        
        return pd.Series({
            'beta': beta,
            'alpha': alpha,
            'exp_age': exp_age,
            'plausible': (exp_age > 0) & (exp_age < row['age']) # Tumor should be younger than patient
        })
    except:
        return pd.Series({
            'beta': np.nan,
            'alpha': np.nan,
            'exp_age': np.nan,
            'plausible': False
        })

In [ ]:
# Apply Hu et al model
original_results = df.apply(calculate_primary_tumor_expansion_age, axis=1)
df = pd.concat([df, original_results], axis=1)
df.head(10)

In [ ]:
# Analyze results
print("Original Model Results:")
print(f"Total cases: {len(df)}")
print(f"Plausible estimates: {df['plausible'].sum()}")
print(f"Implausible estimates: {len(df) - df['plausible'].sum()}")

In [ ]:
trajectories_weighted = calculate_trajectories(df[df['plausible'] == True], age_col='exp_age')
trajectories_filtered = trajectories_weighted[trajectories_weighted['time_step2'] < 0]

# Plotting
plt.figure(figsize=(3.75, 1.5))

# Plot trajectories
for exp_age in df['exp_age'].unique():
    mask = trajectories_filtered['exp_age'] == exp_age
    plt.plot(trajectories_filtered[mask]['time_step2'],
             np.log10(trajectories_filtered[mask]['St']),
             color='#3288bd', alpha=0.6)

plt.axvline(x=0, color='black', linestyle='-')
plt.axvline(x=df[df['plausible'] == True]['exp_age'].mean()*-1, color='black', linestyle='--')
plt.ylim(0, 10)
plt.xlim(-45, 0)

plt.title('Tumor Growth Trajectories')
plt.xlabel('Time relative to diagnosis (years)')
plt.ylabel('log10(Tumor Size)')
plt.grid(True, which='major', axis='both', color='black', alpha=0.1)
plt.show()

In [ ]:
def analyze_sensitivity(df, growth_rate_thresholds):
    """Analyze model sensitivity to different growth rate thresholds"""
    results = []
    
    for threshold in growth_rate_thresholds:
        # Apply threshold
        mask = df['monthly_growth_rate'] >= threshold
        filtered_df = df[mask]
        
        # Calculate metrics
        n_plausible = filtered_df['plausible'].sum()
        n_total = len(filtered_df)
        
        results.append({
            'threshold': threshold,
            'n_cases': n_total,
            'n_plausible': n_plausible,
            'plausible_ratio': n_plausible/n_total if n_total > 0 else 0
        })
    
    return pd.DataFrame(results)

In [ ]:
# Define thresholds to test
thresholds = np.linspace(0, 20, 41)  # 0% to 10% monthly growth rate
df['monthly_growth_rate'] = df['vol_per_change'] / df['time_m']

# Run sensitivity analysis
sensitivity_results = analyze_sensitivity(df, thresholds)

# Plot results
plt.figure(figsize=(7.5, 2.5))
plt.plot(sensitivity_results['threshold'], sensitivity_results['plausible_ratio'], 'o-')
plt.xlabel('Monthly Growth Rate Threshold (%)')
plt.ylabel('Ratio of Plausible Estimates')
plt.title('Model Sensitivity to Growth Rate Threshold')
plt.grid(True)
plt.savefig("../../results/figures/mutational_timing/hu_sensitivity_growth_rate.pdf")
plt.show()

In [ ]:
# 1. Relationship between tumor size and model plausibility
plt.figure(figsize=(7.5, 2.5))
plt.scatter(df[df['plausible']]['tumor_size_t1'], 
           df[df['plausible']]['monthly_growth_rate'],
           label='Plausible', alpha=0.6)
plt.scatter(df[~df['plausible']]['tumor_size_t1'], 
           df[~df['plausible']]['monthly_growth_rate'],
           label='Implausible', alpha=0.6)
plt.xlabel('Initial Tumor Size')
plt.ylabel('Monthly Growth Rate (%)')
plt.title('Tumor Size vs Growth Rate')
plt.legend()
plt.yscale('log')
plt.xscale('log')
plt.grid(True, which='major', color='black', alpha=0.1)
plt.savefig("../../results/figures/mutational_timing/hu_tumor_size_plausibility.pdf")
plt.show()

In [ ]:
# 2. Age distribution analysis
plt.figure(figsize=(12, 6))
sns.kdeplot(data=df[df['plausible']], x='exp_age', label='Estimated Age (Plausible)')
sns.kdeplot(data=df[df['plausible']], x='age', label='Actual Age')
plt.xlabel('Age (years)')
plt.ylabel('Density')
plt.title('Distribution of Estimated vs Actual Ages (Plausible Cases)')
plt.legend()
plt.show()

In [ ]:
# 3. Growth rate distribution
plt.figure(figsize=(12, 6))
sns.boxplot(x='plausible', y='monthly_growth_rate', data=df)
plt.xlabel('Model Estimate Plausibility')
plt.ylabel('Monthly Growth Rate (%)')
plt.title('Growth Rate Distribution by Model Plausibility')
plt.show()

## 3. Alternative Model Implementation
This modeul will directly account for tumor size and growth rate which seem to be major contributors to poor model fit in the original Hu et al model. This is not unexpected as their model was fit to Colon, Breast, and Lung cancer which are more aggressive/rapidly growing than relatively indolent PCC/PGL.

In [ ]:
df = pd.read_excel("../../metadata/PPGL_doubling_time_estimates.xlsx", sheet_name='tumors')
df = df[df['vol_per_change'] > 0.0]

In [ ]:
def calculate_primary_tumor_expansion_age_adjusted(row):
    """Implement the improved model calculations"""
    log_limit_size = np.log(1e11)
    dx_size = row['tumor_size_t1'] * 1e8
    doubling_time_days = row['doubling_time_yr'] * 365
    
    try:
        # Calculate monthly growth rate
        monthly_growth_rate = row['vol_per_change'] / row['time_m']
        
        # Calculate base growth rate
        base_beta = np.log(2) / doubling_time_days
        
        # Adjust beta based on current growth rate and tumor size
        growth_factor = 1 - np.log(dx_size) / log_limit_size
        beta_adj = base_beta * growth_factor * (1 + monthly_growth_rate/100)
        
        # Calculate modified alpha
        alpha_adj = log_limit_size * beta_adj
        
        # Calculate expected age
        exp_age_adj = -1/beta_adj * np.log(1 - np.log(dx_size) * beta_adj / alpha_adj) / 365
        exp_age_adj = max(0, exp_age_adj)
        
        return pd.Series({
            'monthly_growth_rate': monthly_growth_rate,
            'beta': beta_adj,
            'alpha': alpha_adj,
            'exp_age': exp_age_adj,
            'plausible': (exp_age_adj > 0) & (exp_age_adj < row['age'])
        })
    except:
        return pd.Series({
            'monthly_growth_rate': np.nan,
            'beta': np.nan,
            'alpha': np.nan,
            'exp_age': np.nan,
            'plausible': False
        })

In [ ]:
# Apply new model
results = df.apply(calculate_primary_tumor_expansion_age_adjusted, axis=1)
df = pd.concat([df, results], axis=1)

# Compare results
print("Improved Model Results:")
print(f"Total cases: {len(df)}")
print(f"Plausible estimates: {df['plausible'].sum()}")
print(f"Implausible estimates: {len(df) - df['plausible'].sum()}")

In [ ]:
print(np.median(df[df['plausible'] == True]['exp_age']))
print(np.percentile(df[df['plausible'] == True]['exp_age'], 25))
print(np.percentile(df[df['plausible'] == True]['exp_age'], 75))

In [ ]:
trajectories_weighted = calculate_trajectories(df[df['plausible'] == True], age_col='exp_age')
trajectories_filtered = trajectories_weighted[trajectories_weighted['time_step2'] < 0]

# Plotting
plt.figure(figsize=(12, 6))

# Plot trajectories
for exp_age in df['exp_age'].unique():
    mask = trajectories_filtered['exp_age'] == exp_age
    plt.plot(trajectories_filtered[mask]['time_step2'],
             np.log10(trajectories_filtered[mask]['St']),
             color='#3288bd', alpha=0.6)

plt.axvline(x=0, color='black', linestyle='-')
plt.axvline(x=df[df['plausible'] == True]['exp_age'].mean()*-1, color='black', linestyle='--')
plt.ylim(0, 10)

plt.title('Tumor Growth Trajectories')
plt.xlabel('Time relative to diagnosis (years)')
plt.ylabel('log10(Tumor Size)')

plt.grid(True, which='major', axis='both', color='black', alpha=0.1)
plt.savefig("../../results/figures/mutational_timing/primary_tumor_expansion_age.pdf")
plt.show()

In [ ]:
df[df['plausible'] == True]['exp_age'].mean()

## 4. Sensitivity Analysis

In [ ]:
# Define thresholds to test
thresholds = np.linspace(0, 20, 41)  # 0% to 10% monthly growth rate
df['monthly_growth_rate'] = df['vol_per_change'] / df['time_m']

# Run sensitivity analysis
sensitivity_results = analyze_sensitivity(df, thresholds)

# Plot results
plt.figure(figsize=(12, 6))
plt.plot(sensitivity_results['threshold'], sensitivity_results['plausible_ratio'], 'o-')
plt.xlabel('Monthly Growth Rate Threshold (%)')
plt.ylabel('Ratio of Plausible Estimates')
plt.title('Model Sensitivity to Growth Rate Threshold')
plt.grid(True)
plt.show()

In [ ]:
# 1. Relationship between tumor size and model plausibility
plt.figure(figsize=(12, 6))
plt.scatter(df[df['plausible']]['tumor_size_t1'], 
           df[df['plausible']]['monthly_growth_rate'],
           label='Plausible', alpha=0.6)
plt.scatter(df[~df['plausible']]['tumor_size_t1'], 
           df[~df['plausible']]['monthly_growth_rate'],
           label='Implausible', alpha=0.6)
plt.xlabel('Initial Tumor Size')
plt.ylabel('Monthly Growth Rate (%)')
plt.title('Tumor Size vs Growth Rate')
plt.legend()
plt.yscale('log')
plt.xscale('log')
plt.grid(True)
plt.show()

In [ ]:
# 2. Age distribution analysis
plt.figure(figsize=(12, 6))
sns.kdeplot(data=df[df['plausible']], x='exp_age', label='Estimated Age (Plausible)')
sns.kdeplot(data=df[df['plausible']], x='age', label='Actual Age')
plt.xlabel('Age (years)')
plt.ylabel('Density')
plt.title('Distribution of Estimated vs Actual Ages (Plausible Cases)')
plt.legend()
plt.show()

In [ ]:
# 3. Growth rate distribution
plt.figure(figsize=(12, 6))
sns.boxplot(x='plausible', y='monthly_growth_rate', data=df)
plt.xlabel('Model Estimate Plausibility')
plt.ylabel('Monthly Growth Rate (%)')
plt.title('Growth Rate Distribution by Model Plausibility')
plt.show()

## 5. Additional Analyses

In [ ]:
df2 = pd.read_csv("../../metadata/mPPGL_jsi_estimates.csv", index_col=0)
df2.head(10)

In [ ]:
df2['rt_fraction'] = 1 - df2['lmlp']*0.13
#df2 = df2[df2['rt_fraction'] > 0]
df2.head(10)

In [ ]:
df = df[df['plausible'] == True]

In [ ]:
results = []

for i, row in df2.iterrows():
    if np.isnan(row['age_at_diagnosis']):
        continue
        
    ages = df[df['exp_age'] < row['age_at_diagnosis']]['exp_age']
    tmet = row['rt_fraction'] * ages
    tmet = tmet[tmet > -row['time_relapse']]
    results.extend(tmet.tolist())

results = np.array(results) * -1

In [ ]:
# Plot distrubtion of ages
plt.figure(figsize=(5, 5))
sns.histplot(results)
plt.axvline(x=np.median(results), color='black', linestyle='--')
plt.xlabel('Metastatic Seeding')
plt.ylabel('Count')
plt.title('Distribution of Metastatic Seeding Times')
plt.savefig("../../results/figures/mutational_timing/metastatic_seeding.pdf")
plt.show()

In [ ]:
print(np.median(results))
print(np.percentile(results, 25))
print(np.percentile(results, 75))